In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

#### Carga del dataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def load_mnist(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    train_set = torchvision.datasets.MNIST(
        root='./data', 
        train=True,
        download=True, 
        transform=transform
    )
    
    test_set = torchvision.datasets.MNIST(
        root='./data', 
        train=False,
        download=True, 
        transform=transform
    )
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_sizes=[256, 128], output_size=10, activation='relu'):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
        
        for i in range(1, len(hidden_sizes)):
            self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
        
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)
        
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else:
            raise ValueError("Función de activación no soportada")
        
        self.loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Aplanar la imagen
        for layer in self.layers:
            x = self.activation(layer(x))
        x = self.output_layer(x)
        return x
    
    def compute_loss(self, outputs, targets):
        return self.loss_fn(outputs, targets)


#### Entrenamiento y evaluación

In [ ]:
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import ParameterGrid

from skopt import BayesSearchCV
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

In [ ]:
def train_model(model, train_loader, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            outputs = model(data)
            loss = model.compute_loss(outputs, target)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}')


In [7]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Precisión en el conjunto de prueba: {accuracy:.2f}%')
    return accuracy

In [ ]:
def bayesian_hyperparameter_tuning():
    space = [
        Integer(64, 512, name='hidden_size1'),
        Integer(32, 256, name='hidden_size2'),
        Categorical(['relu', 'sigmoid', 'tanh'], name='activation'),
        Real(1e-4, 1e-2, prior='log-uniform', name='lr'),
        Integer(32, 256, name='batch_size'),
        Integer(5, 20, name='epochs')
    ]
    
    @use_named_args(space)
    def objective(**params):
        hidden_sizes = [params['hidden_size1']]
        if params['hidden_size2'] < params['hidden_size1']:
            hidden_sizes.append(params['hidden_size2'])
        
        try:
            train_loader, test_loader = load_mnist(params['batch_size'])
            
            model = MLP(hidden_sizes=hidden_sizes,activation=params['activation']).to(device)
            optimizer = optim.Adam(model.parameters(), lr=params['lr'])
            
            model.train()
            for epoch in range(params['epochs']):
                for batch_idx, (data, target) in enumerate(train_loader):
                    data, target = data.to(device), target.to(device)
                    outputs = model(data)
                    loss = model.compute_loss(outputs, target)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for data, target in test_loader:
                    data, target = data.to(device), target.to(device)
                    outputs = model(data)
                    _, predicted = torch.max(outputs.data, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()
            
            accuracy = 100 * correct / total
            return -accuracy  # minimizamos el negativo de la precisión
            
        except Exception as e:
            print(f"Error con parámetros {params}: {str(e)}")
            return 0  # peor caso posible
    
    print("\nIniciando Bayesian Optimization...")
    result = gp_minimize(
        func=objective,
        dimensions=space,
        n_calls=15,  # iteraciones
        random_state=42,
        verbose=True
    )
    
    print("\nMejores parámetros encontrados:")
    best_params = {
        'hidden_size1': result.x[0],
        'hidden_size2': result.x[1],
        'activation': result.x[2],
        'lr': result.x[3],
        'batch_size': result.x[4],
        'epochs': result.x[5]
    }
    print(best_params)
    print(f"Mejor precisión: {-result.fun:.2f}%")
    
    return best_params

In [ ]:
print("=== Experimento con diferentes configuraciones ===")
configurations = [
    {
        'name': 'Configuración 1 (ReLU, 2 capas)',
        'hidden_sizes': [256, 128],
        'activation': 'relu',
        'lr': 0.001,
        'batch_size': 64,
        'epochs': 10
    },
    {
        'name': 'Configuración 2 (Sigmoid, 3 capas)',
        'hidden_sizes': [512, 256, 128],
        'activation': 'sigmoid',
        'lr': 0.01,
        'batch_size': 128,
        'epochs': 15
    },
    {
        'name': 'Configuración 3 (Tanh, 1 capa)',
        'hidden_sizes': [512],
        'activation': 'tanh',
        'lr': 0.0001,
        'batch_size': 32,
        'epochs': 20
    }
]

results = []

for config in configurations:
    print(f"\nProbando {config['name']}")
    print("-"*50)
    
    train_loader, test_loader = load_mnist(config['batch_size'])
    model = MLP(hidden_sizes=config['hidden_sizes'], activation=config['activation']).to(device)
    optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    
    train_model(model, train_loader, optimizer, config['epochs'])
    accuracy = evaluate_model(model, test_loader)
    
    results.append({
        'Configuración': config['name'],
        'Capas ocultas': str(config['hidden_sizes']),
        'Función de activación': config['activation'],
        'Learning rate': config['lr'],
        'Batch size': config['batch_size'],
        'Épocas': config['epochs'],
        'Precisión (%)': accuracy
    })

df_results = pd.DataFrame(results)
print("\nResultados de todas las configuraciones:")
print(df_results.to_string(index=False))

df_sorted = df_results.sort_values('Precisión (%)', ascending=False)
print("\nRanking de configuraciones:")
print(df_sorted.to_string(index=False))

=== Experimento con diferentes configuraciones ===

Probando Configuración 1 (ReLU, 2 capas)
--------------------------------------------------
Epoch 1/10, Loss: 0.2278
Epoch 2/10, Loss: 0.0935
Epoch 3/10, Loss: 0.0631
Epoch 4/10, Loss: 0.0483
Epoch 5/10, Loss: 0.0392
Epoch 6/10, Loss: 0.0332
Epoch 7/10, Loss: 0.0280
Epoch 8/10, Loss: 0.0248
Epoch 9/10, Loss: 0.0227
Epoch 10/10, Loss: 0.0217
Precisión en el conjunto de prueba: 97.63%

Probando Configuración 2 (Sigmoid, 3 capas)
--------------------------------------------------
Epoch 1/15, Loss: 0.3687
Epoch 2/15, Loss: 0.1723
Epoch 3/15, Loss: 0.1471
Epoch 4/15, Loss: 0.1406
Epoch 5/15, Loss: 0.1312
Epoch 6/15, Loss: 0.1314
Epoch 7/15, Loss: 0.1303
Epoch 8/15, Loss: 0.1275
Epoch 9/15, Loss: 0.1231
Epoch 10/15, Loss: 0.1226
Epoch 11/15, Loss: 0.1225
Epoch 12/15, Loss: 0.1249
Epoch 13/15, Loss: 0.1126
Epoch 14/15, Loss: 0.1133
Epoch 15/15, Loss: 0.1090
Precisión en el conjunto de prueba: 96.25%

Probando Configuración 3 (Tanh, 1 capa)
-

In [13]:
print("\n Bayesian Hyperparameter Tuning ")
best_params = bayesian_hyperparameter_tuning()


 Bayesian Hyperparameter Tuning 

Iniciando Bayesian Optimization...
Iteration No: 1 started. Evaluating function at random point.
Error con parámetros {'hidden_size1': 421, 'hidden_size2': 73, 'activation': 'tanh', 'lr': 0.001562069367563987, 'batch_size': 132, 'epochs': 6}: batch_size should be a positive integer value, but got batch_size=132
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.1842
Function value obtained: 0.0000
Current minimum: 0.0000
Iteration No: 2 started. Evaluating function at random point.
Error con parámetros {'hidden_size1': 270, 'hidden_size2': 107, 'activation': 'relu', 'lr': 0.0020034427927560746, 'batch_size': 45, 'epochs': 16}: batch_size should be a positive integer value, but got batch_size=45
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0630
Function value obtained: 0.0000
Current minimum: 0.0000
Iteration No: 3 started. Evaluating function at random point.
Error con parámetros {'hidden_size1': 484, 'hidde

In [15]:
print("\nEntrenando modelo final con mejores parámetros")
hidden_sizes = [best_params['hidden_size1']]
if 'hidden_size2' in best_params and best_params['hidden_size2'] < best_params['hidden_size1']:
    hidden_sizes.append(best_params['hidden_size2'])

# train_loader, test_loader = load_mnist(best_params['batch_size'])
best_batch_size = int(round(best_params['batch_size']))
train_loader, test_loader = load_mnist(best_batch_size)

model = MLP(hidden_sizes=hidden_sizes, activation=best_params['activation']).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params['lr'])
train_model(model, train_loader, optimizer, best_params['epochs'])
final_accuracy = evaluate_model(model, test_loader)

print("\nResultados Finales")
print(f"Configuración inicial mejor: {df_sorted.iloc[0]['Configuración']} - {df_sorted.iloc[0]['Precisión (%)']:.2f}%")
print(f"Modelo optimizado: {final_accuracy:.2f}% de precisión")


Entrenando modelo final con mejores parámetros
Epoch 1/6, Loss: 0.2546
Epoch 2/6, Loss: 0.1038
Epoch 3/6, Loss: 0.0719
Epoch 4/6, Loss: 0.0578
Epoch 5/6, Loss: 0.0520
Epoch 6/6, Loss: 0.0424
Precisión en el conjunto de prueba: 97.57%

Resultados Finales
Configuración inicial mejor: Configuración 3 (Tanh, 1 capa) - 98.19%
Modelo optimizado: 97.57% de precisión


#### Comparación de los resultados
Los experimentos mostraron diferencias significativas en el rendimiento según la configuración de hiperparámetros. La Configuración 3 (Tanh, 1 capa) obtuvo el mejor rendimiento con un 98.19% de precisión, seguida de la Configuración 1 (ReLU, 2 capas) con 97.63%, mientras que la Configuración 2 (Sigmoid, 3 capas) alcanzó solo 96.25%. El modelo optimizado mediante búsqueda bayesiana logró un 97.57%, ligeramente por debajo de las mejores configuraciones manuales debido a problemas técnicos durante la optimización.

#### Hiperparámetros más influyentes
La función de activación demostró ser crítica: Tanh superó a ReLU y Sigmoid debido a su capacidad para mantener gradientes estables en redes poco profundas. En cuanto a la arquitectura de la red, la configuración más simple (1 capa oculta con 512 neuronas) superó a diseños más complejos, sugiriendo que MNIST no requiere redes profundas para alcanzar alto rendimiento. El learning rate mostró que valores pequeños (0.0001-0.001) permiten mejor convergencia que valores altos (0.01), que causan inestabilidad en el entrenamiento.

Los tamaños de batch pequeños (32-64) proporcionaron mejor generalización que batches grandes (128-132), ya que el ruido en los gradientes ayuda a evitar mínimos locales. El número de épocas confirmó que un entrenamiento más prolongado (20 épocas) permite mejor convergencia, especialmente con learning rates pequeños, aunque requiere mayor tiempo de cómputo.

#### Problemas con la optimización bayesiana
La optimización automática no logró superar las configuraciones manuales debido a dos factores principales: (1) errores técnicos en el manejo de valores enteros para el batch_size, y (2) número insuficiente de iteraciones para explorar adecuadamente el espacio de búsqueda. Esto resalta la importancia de preparar adecuadamente el proceso de optimización y validar los rangos de parámetros.